In [2]:
import pandas as pd


file_mappings = {
    2020: {
        "file": "../../data/assessment_csv/2020.csv",
        "last_name_col": "Last Name",
        "first_name_col": "First Name",
        "entrance_col": "Writing",
        "exit_col": "Exit Writing"
    },
    2019: {
        "file": "../../data/assessment_csv/2019.csv",
        "last_name_col": "Last Name",
        "first_name_col": "First Name",
        "entrance_col": "Writing Entrance",
        "exit_col": "Writing test Final "
    },
    2018: {
        "file": "../../data/assessment_csv/2018.csv",
        "last_name_col": "Last Name",
        "first_name_col": "First Name",
        "entrance_col": "Writing Entrance",
        "exit_col": "Writing test Final "
    },
    2017: {
        "file": "../../data/assessment_csv/2017.csv",
        "last_name_col": "Last Name",
        "first_name_col": "First Name",
        "entrance_col": "Writing Entrance",
        "exit_col": "Writing test Final "
    }
}



In [3]:

all_marks_dfs = []

for year, mapping in file_mappings.items():
    file_name = mapping["file"]
    
    df = pd.read_csv(file_name, sep='\t')
    
    col_mapping = {
        mapping["last_name_col"]: "Last Name",
        mapping["first_name_col"]: "First Name",
        mapping["entrance_col"]: "Entrance",
        mapping["exit_col"]: "Exit"
    }
    
    df_selected = df[list(col_mapping.keys())].rename(columns=col_mapping)
    
    df_selected["Year"] = year
    
    all_marks_dfs.append(df_selected)


final_marks_df = pd.concat(all_marks_dfs, ignore_index=True)

final_marks_df.to_csv("merged_raw_marks_step1.csv", index=False)


In [ ]:

print(final_marks_df.info())

```<class 'pandas.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Last Name   493 non-null    str  
 1   First Name  493 non-null    str  
 2   Entrance    386 non-null    str  
 3   Exit        473 non-null    str  
 4   Year        493 non-null    int64
dtypes: int64(1), str(4)
memory usage: 19.4 KB
None
```

In [ ]:
import pandas as pd
import numpy as np
import re

df_marks = pd.read_csv("./preprocessed/merged_raw_marks_step1.csv")


# --- Anonymization  ---
df_marks['Student_Key'] = df_marks['Last Name'].str.strip().str.upper() + '_' + df_marks['First Name'].str.strip().str.upper()

# Getting the unique keys and assigning sequential Student_IDs
unique_students = df_marks[['Student_Key']].drop_duplicates().reset_index(drop=True)
unique_students['Student_ID'] = 'STUDENT_' + (unique_students.index + 1).astype(str).str.zfill(4)

# Creating the final lookup table (Last Name, First Name, Student_ID)
anonymization_lookup = df_marks[['Last Name', 'First Name', 'Student_Key']].drop_duplicates()
anonymization_lookup = pd.merge(anonymization_lookup, unique_students, on='Student_Key')


In [ ]:
anonymization_lookup.info()

```
<class 'pandas.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Last Name    472 non-null    str  
 1   First Name   472 non-null    str  
 2   Student_Key  472 non-null    str  
 3   Student_ID   472 non-null    str  
dtypes: str(4)
memory usage: 14.9 KB
```

In [8]:
anonymization_lookup.to_csv('anonymization_lookup_upd.csv', index=False)

In [12]:
mappings_from_mid_to_actfl = {

'NL': 0.0, 'NL+': '0.0+',

'NM-': '0.5-', 'NM': 0.5, 'NM+': '0.5+',

'NH-': '0.9-', 'NH': 0.9, 'NH+': '0.9+',

'IL-': '1.0-', 'IL': 1.0, 'IL+': '1.0+',

'IM-': '1.5=', 'IM': 1.5, 'IM+': '1.5+',

'IH-': '1.9+', 'IH': 1.9,'IH+': '1.9+',

'AL-': '2.0-', 'AL': 2, 'AL+': '2.0+',

'AM-': '2.5-', 'AM': 2.5, 'AM+': '2.5+',

'AH-': '2.9+', 'AH': 2.9, 'AH+': '2.9+',

'S': 3.0,

'NO.TEST': np.nan, 'NOTEST': np.nan, 'NOSHOW': np.nan, 'NT': np.nan,

}

In [15]:
import pandas as pd
import numpy as np
import re

# --- Score Standardization to 27-Point Ordinal Scale (0-24 Index) ---

# Master mapping dictionary: Raw Score/Label (Key) -> Ordinal Index (Value)
# Note: The mapping below omits NL levels (0.0/0.0+) as they are the very lowest and are sparse.
# The scale starts at NM- (Index 0). The total range is 0 to 24 (25 distinct values).

# 1. Define the core ACTFL labels and their corresponding starting ordinal index
actfl_sub_levels = [
    ('NL', 0), ('NM', 1), ('NH', 2), 
    ('IL', 3), ('IM', 4), ('IH', 5), 
    ('AL', 6), ('AM', 7), ('AH', 8), 
    ('S', 9)
]

granular_map = {}

# 2. Map ACTFL Labels (e.g., 'IM-', 'IM', 'IM+') to sequential indices
for label, start_index in actfl_sub_levels:
        # Map sub-levels: X- => index, X => index+1, X+ => index+2
        granular_map[f'{label}-'] = start_index 
        granular_map[label] = start_index 
        granular_map[f'{label}+'] = start_index
        


In [64]:
granular_map

{'NL-': nan,
 'NL': nan,
 'NL+': nan,
 'NM-': 1,
 'NM': 1,
 'NM+': 1,
 'NH-': 2,
 'NH': 2,
 'NH+': 2,
 'IL-': 3,
 'IL': 3,
 'IL+': 3,
 'IM-': 4,
 'IM': 4,
 'IM+': 4,
 'IH-': 5,
 'IH': 5,
 'IH+': 5,
 'AL-': 6,
 'AL': 6,
 'AL+': 6,
 'AM-': 7,
 'AM': 7,
 'AM+': 7,
 'AH-': 8,
 'AH': 8,
 'AH+': 8,
 'S-': 9,
 'S': 9,
 'S+': 9,
 '0.0-': 0,
 '0.0': 0,
 '0.0+': 0,
 '0.5-': 1,
 '0.5': 1,
 '0.5+': 1,
 '0.9-': 2,
 '0.9': 2,
 '0.9+': 2,
 '1.0-': 3,
 '1.0': 3,
 '1.0+': 3,
 '1.5-': 4,
 '1.5': 4,
 '1.5+': 14,
 '1.9-': 5,
 '1.9': 5,
 '1.9+': 5,
 '2.0-': 6,
 '2.0': 6,
 '2.0+': 6,
 '2.5-': 7,
 '2.5': 7,
 '2.5+': 7,
 '2.9-': 8,
 '2.9': 8,
 '2.9+': 8,
 '3.0-': 9,
 '3.0': 9,
 '3.0+': 9,
 '0,0-': 0,
 '0,0': 0,
 '0,0+': 0,
 '0,5-': 1,
 '0,5': 1,
 '0,5+': 1,
 '0,9-': 2,
 '0,9': 2,
 '0,9+': 2,
 '1,0-': 3,
 '1,0': 3,
 '1,0+': 3,
 '1,5-': 4,
 '1,5': 4,
 '1,5+': 14,
 '1,9-': 5,
 '1,9': 5,
 '1,9+': 5,
 '2,0-': 6,
 '2,0': 6,
 '2,0+': 6,
 '2,5-': 7,
 '2,5': 7,
 '2,5+': 7,
 '2,9-': 8,
 '2,9': 8,
 '2,9+': 8,
 '3,0-': 9

In [ ]:
# 3. Map Raw Numeric Strings to the same index
# Note: We must handle comma/dot and the special '=' sign (from '1.5=')

# ACTFL Base Numeric Points: 0.5, 0.9, 1.0, 1.5, 1.9, 2.0, 2.5, 2.9, 3.0
base_numeric_map = {
    '0.0-': 0, '0.0': 0, '0.0+': 0,
    '0.5-': 1, '0.5': 1, '0.5+': 1, # NM base
    '0.9-': 2, '0.9': 2, '0.9+': 2, # NH base
    '1.0-': 3, '1.0': 3, '1.0+': 3, # IL base
    '1.5-': 4, '1.5': 4, '1.5+': 4, # IM base
    '1.9-': 5, '1.9': 5, '1.9+': 5, # IH base
    '2.0-': 6, '2.0': 6, '2.0+': 6, # AL base
    '2.5-': 7, '2.5': 7, '2.5+': 7, # AM base
    '2.9-': 8, '2.9': 8, '2.9+': 8, # AH base
    '3.0-': 9, '3.0': 9, '3.0+': 9
}

# Updating granular_map with these numeric-string mappings
granular_map.update(base_numeric_map)

# Adding numeric keys with comma decimals (e.g., "1,5" -> 10)
granular_map_commas = {k.replace('.', ','): v for k, v in base_numeric_map.items() if '.' in k}
granular_map.update(granular_map_commas)


In [66]:
granular_map

{'NL-': nan,
 'NL': nan,
 'NL+': nan,
 'NM-': 1,
 'NM': 1,
 'NM+': 1,
 'NH-': 2,
 'NH': 2,
 'NH+': 2,
 'IL-': 3,
 'IL': 3,
 'IL+': 3,
 'IM-': 4,
 'IM': 4,
 'IM+': 4,
 'IH-': 5,
 'IH': 5,
 'IH+': 5,
 'AL-': 6,
 'AL': 6,
 'AL+': 6,
 'AM-': 7,
 'AM': 7,
 'AM+': 7,
 'AH-': 8,
 'AH': 8,
 'AH+': 8,
 'S-': 9,
 'S': 9,
 'S+': 9,
 '0.0-': 0,
 '0.0': 0,
 '0.0+': 0,
 '0.5-': 1,
 '0.5': 1,
 '0.5+': 1,
 '0.9-': 2,
 '0.9': 2,
 '0.9+': 2,
 '1.0-': 3,
 '1.0': 3,
 '1.0+': 3,
 '1.5-': 4,
 '1.5': 4,
 '1.5+': 4,
 '1.9-': 5,
 '1.9': 5,
 '1.9+': 5,
 '2.0-': 6,
 '2.0': 6,
 '2.0+': 6,
 '2.5-': 7,
 '2.5': 7,
 '2.5+': 7,
 '2.9-': 8,
 '2.9': 8,
 '2.9+': 8,
 '3.0-': 9,
 '3.0': 9,
 '3.0+': 9,
 '0,0-': 0,
 '0,0': 0,
 '0,0+': 0,
 '0,5-': 1,
 '0,5': 1,
 '0,5+': 1,
 '0,9-': 2,
 '0,9': 2,
 '0,9+': 2,
 '1,0-': 3,
 '1,0': 3,
 '1,0+': 3,
 '1,5-': 4,
 '1,5': 4,
 '1,5+': 4,
 '1,9-': 5,
 '1,9': 5,
 '1,9+': 5,
 '2,0-': 6,
 '2,0': 6,
 '2,0+': 6,
 '2,5-': 7,
 '2,5': 7,
 '2,5+': 7,
 '2,9-': 8,
 '2,9': 8,
 '2,9+': 8,
 '3,0-': 9,


In [68]:
granular_map_df = pd.DataFrame(list(granular_map.items()), columns=['key', 'value'])
granular_map_df.to_csv('granular_marks_upd')

In [69]:
print(granular_map)

{'NL-': nan, 'NL': nan, 'NL+': nan, 'NM-': 1, 'NM': 1, 'NM+': 1, 'NH-': 2, 'NH': 2, 'NH+': 2, 'IL-': 3, 'IL': 3, 'IL+': 3, 'IM-': 4, 'IM': 4, 'IM+': 4, 'IH-': 5, 'IH': 5, 'IH+': 5, 'AL-': 6, 'AL': 6, 'AL+': 6, 'AM-': 7, 'AM': 7, 'AM+': 7, 'AH-': 8, 'AH': 8, 'AH+': 8, 'S-': 9, 'S': 9, 'S+': 9, '0.0-': 0, '0.0': 0, '0.0+': 0, '0.5-': 1, '0.5': 1, '0.5+': 1, '0.9-': 2, '0.9': 2, '0.9+': 2, '1.0-': 3, '1.0': 3, '1.0+': 3, '1.5-': 4, '1.5': 4, '1.5+': 4, '1.9-': 5, '1.9': 5, '1.9+': 5, '2.0-': 6, '2.0': 6, '2.0+': 6, '2.5-': 7, '2.5': 7, '2.5+': 7, '2.9-': 8, '2.9': 8, '2.9+': 8, '3.0-': 9, '3.0': 9, '3.0+': 9, '0,0-': 0, '0,0': 0, '0,0+': 0, '0,5-': 1, '0,5': 1, '0,5+': 1, '0,9-': 2, '0,9': 2, '0,9+': 2, '1,0-': 3, '1,0': 3, '1,0+': 3, '1,5-': 4, '1,5': 4, '1,5+': 4, '1,9-': 5, '1,9': 5, '1,9+': 5, '2,0-': 6, '2,0': 6, '2,0+': 6, '2,5-': 7, '2,5': 7, '2,5+': 7, '2,9-': 8, '2,9': 8, '2,9+': 8, '3,0-': 9, '3,0': 9, '3,0+': 9, 'GROUP': nan, 'NO.TEST': nan, 'NOTEST': nan, 'NOSHOW': nan, 'NT': 

In [70]:

# Add other non-score labels
granular_map.update({
    'NL-': np.nan, 'NL': np.nan, 'NL+': np.nan, # Excluding NL from the scale (mapped to NaN)
    'GROUP': np.nan, 'NO.TEST': np.nan, 'NOTEST': np.nan, 'NOSHOW': np.nan, 'NT': np.nan
})

In [71]:
granular_map

{'NL-': nan,
 'NL': nan,
 'NL+': nan,
 'NM-': 1,
 'NM': 1,
 'NM+': 1,
 'NH-': 2,
 'NH': 2,
 'NH+': 2,
 'IL-': 3,
 'IL': 3,
 'IL+': 3,
 'IM-': 4,
 'IM': 4,
 'IM+': 4,
 'IH-': 5,
 'IH': 5,
 'IH+': 5,
 'AL-': 6,
 'AL': 6,
 'AL+': 6,
 'AM-': 7,
 'AM': 7,
 'AM+': 7,
 'AH-': 8,
 'AH': 8,
 'AH+': 8,
 'S-': 9,
 'S': 9,
 'S+': 9,
 '0.0-': 0,
 '0.0': 0,
 '0.0+': 0,
 '0.5-': 1,
 '0.5': 1,
 '0.5+': 1,
 '0.9-': 2,
 '0.9': 2,
 '0.9+': 2,
 '1.0-': 3,
 '1.0': 3,
 '1.0+': 3,
 '1.5-': 4,
 '1.5': 4,
 '1.5+': 4,
 '1.9-': 5,
 '1.9': 5,
 '1.9+': 5,
 '2.0-': 6,
 '2.0': 6,
 '2.0+': 6,
 '2.5-': 7,
 '2.5': 7,
 '2.5+': 7,
 '2.9-': 8,
 '2.9': 8,
 '2.9+': 8,
 '3.0-': 9,
 '3.0': 9,
 '3.0+': 9,
 '0,0-': 0,
 '0,0': 0,
 '0,0+': 0,
 '0,5-': 1,
 '0,5': 1,
 '0,5+': 1,
 '0,9-': 2,
 '0,9': 2,
 '0,9+': 2,
 '1,0-': 3,
 '1,0': 3,
 '1,0+': 3,
 '1,5-': 4,
 '1,5': 4,
 '1,5+': 4,
 '1,9-': 5,
 '1,9': 5,
 '1,9+': 5,
 '2,0-': 6,
 '2,0': 6,
 '2,0+': 6,
 '2,5-': 7,
 '2,5': 7,
 '2,5+': 7,
 '2,9-': 8,
 '2,9': 8,
 '2,9+': 8,
 '3,0-': 9,


In [ ]:



def standardize_to_ordinal(score):
    """Converts a raw score string/value to the unified ordinal numeric scale (0-24)."""
    if pd.isna(score):
        return np.nan
    
    s = str(score).strip().upper()
    
    # Direct mapping for labels and fixed numeric strings
    if s in granular_map:
        return granular_map[s]
    
    # Handle simple integer/float scores (e.g., '2' or '1.5') not explicitly in map
    try:
        num_val = float(s)
        # Check if this simple number corresponds to a base ACTFL score
        if num_val == 0.9: return granular_map['0.9']
        if num_val == 1.0: return granular_map['1.0']
        if num_val == 1.5: return granular_map['1.5']
        if num_val == 1.9: return granular_map['1.9']
        if num_val == 2.0: return granular_map['2.0']
        if num_val == 2.5: return granular_map['2.5']
        if num_val == 2.9: return granular_map['2.9']
        if num_val == 3.0: return granular_map['3.0']
    except ValueError:
        pass 

    return np.nan


In [73]:

# Apply the new standardization function
df_marks['Entrance_Ordinal'] = df_marks['Entrance'].apply(standardize_to_ordinal)
df_marks['Exit_Ordinal'] = df_marks['Exit'].apply(standardize_to_ordinal)


In [ ]:


df_anonymized = df_marks.merge(unique_students, on='Student_Key')[['Year', 'Student_ID', 'Entrance_Ordinal', 'Exit_Ordinal']]


In [ ]:
print("\n--- Standardized Ordinal Score Value Counts (Entrance) ---")
# Only show the top 15 non-NaN values
print(df_anonymized['Entrance_Ordinal'].value_counts(dropna=False).sort_index().head(15))

```
<class 'pandas.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Last Name    472 non-null    str  
 1   First Name   472 non-null    str  
 2   Student_Key  472 non-null    str  
 3   Student_ID   472 non-null    str  
dtypes: str(4)
memory usage: 14.9 KB
```

In [ ]:

df_anonymized.to_csv("upd_preprocessed/upd_merged_anonymized_marks_ordinal_step2.csv", index=False)

In [78]:
anonymization_lookup.to_csv("upd_preprocessed/upd_anonymization_lookup_table.csv", index=False)

In [79]:
import pandas as pd
import numpy as np

df_marks_raw = pd.read_csv("upd_preprocessed/upd_merged_anonymized_marks_ordinal_step2.csv")


In [80]:
# This converts the wide format (one row, two score columns) to a long format (two rows, one score column)
df_long = pd.melt(
    df_marks_raw,
    id_vars=['Year', 'Student_ID'],
    value_vars=['Entrance_Ordinal', 'Exit_Ordinal'],
    var_name='Test_type_full',
    value_name='Mark_Ordinal'
)


In [81]:
df_long['Test_type'] = df_long['Test_type_full'].str.replace('_Ordinal', '', regex=False)

In [82]:
df_long

,Year,Student_ID,Test_type_full,Mark_Ordinal,Test_type
0,2020,STUDENT_0001,Entrance_Ordinal,NaN,Entrance
1,2020,STUDENT_0002,Entrance_Ordinal,NaN,Entrance
2,2020,STUDENT_0003,Entrance_Ordinal,NaN,Entrance
3,2020,STUDENT_0004,Entrance_Ordinal,NaN,Entrance
4,2020,STUDENT_0005,Entrance_Ordinal,NaN,Entrance
...,...,...,...,...,...
981,2017,STUDENT_0469,Exit_Ordinal,NaN,Exit
982,2017,STUDENT_0470,Exit_Ordinal,3.0,Exit
983,2017,STUDENT_0136,Exit_Ordinal,4.0,Exit
984,2017,STUDENT_0471,Exit_Ordinal,5.0,Exit


In [ ]:


# Dropping rows with missing scores (exams that were not taken/scored)
df_final = df_long.dropna(subset=['Mark_Ordinal']).copy()

# Selecting final columns and ensure Mark_Ordinal is integer (since it's an ordinal scale)
df_final = df_final[['Year', 'Student_ID', 'Test_type', 'Mark_Ordinal']]
df_final['Mark_Ordinal'] = df_final['Mark_Ordinal']


In [ ]:

# 5. Saving the resulting DataFrame
output_file = "upd_preprocessed/upd_marks_for_each_essay_step3.csv"
df_final.to_csv(output_file, index=False)

In [85]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:
file_paths = []
with open("../../data/preprocessed/files.txt", 'r') as f:
    file_paths = [
        line.strip().replace('./preprocessed/', '').lstrip('./')
        for line in f if line.strip().endswith('.txt')
    ]
file_paths

In [87]:
df_lookup = pd.read_csv("upd_preprocessed/upd_anonymization_lookup_table.csv")

In [88]:
df_scores = pd.read_csv("upd_preprocessed/upd_marks_for_each_essay_step3.csv")
df_scores

,Year,Student_ID,Test_type,Mark_Ordinal
0,2020,STUDENT_0026,Entrance,4.0
1,2020,STUDENT_0027,Entrance,4.0
2,2020,STUDENT_0028,Entrance,3.0
3,2020,STUDENT_0029,Entrance,3.0
4,2020,STUDENT_0030,Entrance,3.0
...,...,...,...,...
826,2017,STUDENT_0468,Exit,4.0
827,2017,STUDENT_0470,Exit,3.0
828,2017,STUDENT_0136,Exit,4.0
829,2017,STUDENT_0471,Exit,5.0


In [ ]:
def parse_essay_path_and_key(path):
    """
    Parses essay file path to extract Student_Key (LASTNAME_FIRSTNAME), 
    Year, and Test_type ('Entrance' or 'Exit').
    """
    
    # 1. Extracting Year from the path structure
    year_match = re.search(r'(2017|2018|2019|2020)', path)
    year = int(year_match.group(0)) if year_match else None

    # 2. Extracting filename and the name+test_type part
    filename = os.path.basename(path)
    
    # Pattern to capture name parts (e.g., Name_Surname) and the test type (Entry/Exit)
    # This assumes the file is ALWAYS First_Name_Last_Name
    name_type_match = re.search(r'([A-Za-z]+_[A-Za-z]+).*?(Entry|Exit)\d', filename, re.IGNORECASE)
    
    student_key = None
    test_type = None

    if name_type_match:
        # File is First_Name_Last_Name 
        name_parts = name_type_match.group(1).split('_')
        
        if len(name_parts) == 2:
            first_name = name_parts[0]
            last_name = name_parts[1]
            
            # CRITICAL: Recreate the Student_Key in the standard format: LASTNAME_FIRSTNAME (UPPERCASE)
            student_key = f"{last_name.strip().upper()}_{first_name.strip().upper()}"
        
        # Test type extraction (e.g., 'Entry' -> 'Entrance', 'Exit' -> 'Exit')
        test_type_raw = name_type_match.group(2)
        test_type = 'Entrance' if 'Entr' in test_type_raw else 'Exit' if 'Exit' in test_type_raw else None

    return {
        'essay_path': path,
        'Year': year,
        'Test_type': test_type,  # Matches column name in df_scores
        'Student_Key': student_key
    }

In [ ]:
df_essays_raw = pd.DataFrame([parse_essay_path_and_key(path) for path in file_paths])
df_essays_raw = df_essays_raw.dropna(subset=['Student_Key', 'Test_type', 'Year']) 


df_essays_merged = pd.merge(
    df_essays_raw,
    df_lookup[['Student_Key', 'Student_ID']].drop_duplicates(), 
    on='Student_Key',
    how='left'
)
df_essays_merged = df_essays_merged.dropna(subset=['Student_ID'])
df_essays_merged


In [ ]:
df_train = pd.merge(
    df_essays_merged,
    df_scores,
    on=['Student_ID', 'Year', 'Test_type'],
    how='inner'
)
df_train

In [92]:
def read_essay_content(path):
    """Reads the content of an essay file, handling common encodings for Russian text."""
    # Prioritize 'utf-8', but fall back to 'cp1251' or ignore errors if necessary
    # 'errors="ignore"' is safest for varied archival data
    try:
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            return f.read().strip()
    except Exception as e:
        print(f"Error reading file {path}: {e}")
        return None

In [ ]:
def parse_essay_path_and_key(path):
    """
    Parses essay file path to extract Student_Key (LASTNAME_FIRSTNAME), 
    Year, Test_type, and the raw name parts.
    """
    
    # 1. Extract Year from the path structure
    year_match = re.search(r'(2017|2018|2019|2020)', path)
    year = int(year_match.group(0)) if year_match else None

    # 2. Extract filename and the name+test_type part (e.g., Name_Surname_Entry2)
    filename = os.path.basename(path)
    
    # Pattern to capture name parts (e.g. Name_Surname) and the test type (Entry/Exit)
    # The 'Entry'/'Exit' part anchors the identification of the name
    name_type_match = re.search(r'([A-Za-z]+_[A-Za-z]+).*?(Entry|Exit)\d', filename, re.IGNORECASE)
    
    student_key = None
    test_type = None

    if name_type_match:
        # Filename format is confirmed to be First_Name_Last_Name
        name_parts = name_type_match.group(1).split('_')
        
        if len(name_parts) == 2:
            first_name_raw = name_parts[0]
            last_name_raw = name_parts[1]
            
            # CRITICAL: Recreating the Student_Key in the required standard format: LASTNAME_FIRSTNAME (UPPERCASE)
            # This ensures matching with the lookup table.
            student_key = f"{last_name_raw.strip().upper()}_{first_name_raw.strip().upper()}"
        
        # Test type extraction (e.g., 'Entry' -> 'Entrance', 'Exit' -> 'Exit')
        test_type_raw = name_type_match.group(2)
        test_type = 'Entrance' if 'Entr' in test_type_raw else 'Exit' if 'Exit' in test_type_raw else None

    return {
        'essay_path': path,
        'Year': year,
        'Test_type': test_type,
        'Student_Key': student_key
    }



In [ ]:
# --- 2. Loading and Process Data ---

# Load all essay file paths
with open("../../data/preprocessed/files.txt", 'r') as f:
    file_paths = [line.strip().split('./Transcribed ORIGINAL data txt Summer 2017 2018 2019 2020 (1)')[-1] for line in f if line.strip().endswith('.txt')]

file_paths

In [ ]:
# Load the lookup table (Name -> ID)
df_lookup = pd.read_csv("upd_preprocessed/upd_anonymization_lookup_table.csv")
df_lookup

In [96]:
# Load the prepared scores table (ID, Year, Test_type -> Score)
df_scores = pd.read_csv("upd_preprocessed/upd_marks_for_each_essay_step3.csv")
df_scores['Mark_Ordinal'] = df_scores['Mark_Ordinal'].astype(float) # Ensure types match for merge
df_scores

,Year,Student_ID,Test_type,Mark_Ordinal
0,2020,STUDENT_0026,Entrance,4.0
1,2020,STUDENT_0027,Entrance,4.0
2,2020,STUDENT_0028,Entrance,3.0
3,2020,STUDENT_0029,Entrance,3.0
4,2020,STUDENT_0030,Entrance,3.0
...,...,...,...,...
826,2017,STUDENT_0468,Exit,4.0
827,2017,STUDENT_0470,Exit,3.0
828,2017,STUDENT_0136,Exit,4.0
829,2017,STUDENT_0471,Exit,5.0


In [ ]:

# 3. Create Essay Metadata DataFrame
df_essays_raw = pd.DataFrame([parse_essay_path_and_key(path) for path in file_paths])
df_essays_raw = df_essays_raw.dropna(subset=['Student_Key', 'Test_type'])
df_essays_raw

In [ ]:
df_essays_with_id = pd.merge(
    df_essays_raw,
    df_lookup[['Student_Key', 'Student_ID']].drop_duplicates(), 
    on='Student_Key',
    how='left'
)
df_essays_with_id

In [99]:
df_essays_with_id.to_csv('upd_preprocessed/upd_essays_with_id.csv', index=False)
df_scores.to_csv('upd_preprocessed/upd_scores.csv', index=False)

In [ ]:
df_train = pd.merge(
    df_essays_with_id,
    df_scores,
    on=['Student_ID', 'Year', 'Test_type'],
    how='inner' 
)
df_train

In [101]:
df_train.to_csv('upd_preprocessed/upd_training_data_wo_texts.csv', index=False)

In [102]:
import pandas as pd
import numpy as np
import re
import os


def read_essay_content(p):
    """Reads the content of an essay file, handling common encodings for Russian text."""
    # Prioritize 'utf-8', but fall back to 'cp1251' or ignore errors if necessary
    # 'errors="ignore"' is safest for varied archival data
    path = '../../data/preprocessed/Transcribed ORIGINAL data txt Summer 2017 2018 2019 2020 (1)/'+p
    try:
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            return f.read().strip()
    except Exception as e:
        print(f"Error reading file {path}: {e}")
        return None

In [104]:
BERT_TEXT_COLUMN = 'essay_text'

In [105]:
df_final = pd.read_csv("upd_preprocessed/upd_training_data_wo_texts.csv")

In [106]:
df_final[BERT_TEXT_COLUMN] = df_final['essay_path'].apply(read_essay_content)

In [107]:
df_final.to_csv('upd_preprocessed/training_data.csv', index=False, sep='\t')

In [108]:
df_training_baseline = pd.DataFrame()
df_training_baseline['essay'] = df_final['essay_text']
df_training_baseline['score'] = df_final['Mark_Ordinal']
df_training_baseline.to_csv('upd_preprocessed/upd_training.csv', index=False)
df_training_baseline.to_csv('training_data/upd_training.csv', index=False)
df_training_baseline

,essay,score
0,Сообщение. Об этом слов мы думаем каждый день....,5.0
1,Я не знаю.,4.0
2,У меня не есть словай...,2.0
3,"""Следуюший, пожалуйста.""\nЕсли мне не нравится...",4.0
4,"Мне кажется, что личные телефоны и компютери о...",5.0
...,...,...
1133,"Привет, ребята! \nПрограмма почти закончилась....",7.0
1134,Мой самый интересный день в Миддлбери произоше...,7.0
1135,Мой университет называется Лафайетте. Он наход...,7.0
1136,"Честность – важное качество в жизни, но всегда...",7.0
